In [1]:
suppressPackageStartupMessages(suppressWarnings({
library(rhdf5)
library(Matrix)
}))

In [5]:
prj.dir <- "/scratch/gpfs/KANG/sereno/spatialstem"
src.dir <- file.path(prj.dir, "sourcefiles")
raw.dir <- file.path(src.dir, "raw")
had.dir <- file.path(src.dir, "h5ad")
int.dir <- file.path(prj.dir, "intermediates")
fig.dir <- file.path(prj.dir, "figs")
csv.dir <- file.path(src.dir, "csv")
exp.dir <- file.path(csv.dir, "exp")
cor.dir <- file.path(csv.dir, "coords")

In [9]:
# Function for opening an h5ad file as a formatted 10x object.
exp.files <- list.files(exp.dir)
exp.paths <- paste0(exp.dir, "/", exp.files)

In [12]:
exp.paths[1]

[1] "/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/csv/exp/p01_fetalliver_exp.csv"

[1] "/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/csv/exp"